In [1]:
import os

In [2]:
%pwd

'/Users/abdulrasheed/Desktop/ImageClassification/research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [5]:
from source.CNNClassifier.constant.__init_ import CONFIG_FILE_PATH, PARAMS_FILE_PATH

from source.CNNClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config             


In [7]:
### Components

In [8]:
import os
import urllib.request as request
import zipfile  
from source.CNNClassifier import logger
from source.CNNClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file

            )
            logger.info(f"{filename} download! with following info: \n{header}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path : str
        extract zip file to data diretory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        print(f"File path: {self.config.local_data_file}")

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}") 


In [10]:
### pipeline

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    logger.error(f"Error during data ingestion: {str(e)}")

[2024-11-27 16:14:45,103: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-27 16:14:45,106: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-27 16:14:45,107: INFO: common: created directory at: artifacts]
[2024-11-27 16:14:45,108: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-27 16:14:45,109: INFO: 2873370200: File already exists of size: ~ 282 KB]
File path: artifacts/data_ingestion/data.zip


BadZipFile: File is not a zip file